In [1]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
import os
import glob 

In [2]:
filter_L = ['L13']

In [3]:
# run in about 3 minutes in the first time installed
if torch.cuda.is_available():
    device = "cuda"  # Use GPU with CUDA
elif torch.backends.mps.is_available():
    device = "mps"  # Use Metal Performance Shaders for Apple Silicon
else:
    device = "cpu"  # Default to CPU
print(f"Using: {device}")
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")

Using: mps


/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# distinct videos
batch_videos_dict = {}
path_pattern = os.path.join(os.getcwd(), '..', '..', 'data', 
                            'batch*', 'keyframes', '*', '*')
paths = glob.glob(path_pattern)

for path in paths:
    _, batch_name, _, _, video_name = path.rsplit(os.sep, 4)
    if batch_name not in batch_videos_dict:
        batch_videos_dict[batch_name] = []
    batch_videos_dict[batch_name].append(video_name)

for batch in batch_videos_dict.keys():
    batch_videos_dict[batch].sort()


['L13_V001',
 'L13_V002',
 'L13_V003',
 'L13_V004',
 'L13_V005',
 'L13_V006',
 'L13_V007',
 'L13_V008',
 'L13_V009',
 'L13_V010',
 'L13_V011',
 'L13_V012',
 'L13_V013',
 'L13_V014',
 'L13_V015',
 'L13_V016',
 'L13_V017',
 'L13_V018',
 'L13_V019',
 'L13_V020',
 'L13_V021',
 'L13_V022',
 'L13_V023',
 'L13_V024',
 'L13_V025',
 'L13_V026',
 'L13_V027',
 'L13_V028',
 'L13_V029',
 'L13_V030',
 'L14_V001',
 'L14_V002',
 'L14_V003',
 'L14_V004',
 'L14_V005',
 'L14_V006',
 'L14_V007',
 'L14_V008',
 'L14_V009',
 'L14_V011',
 'L14_V012',
 'L14_V013',
 'L14_V014',
 'L14_V015',
 'L14_V016',
 'L14_V017',
 'L14_V018',
 'L14_V019',
 'L14_V020',
 'L14_V021',
 'L14_V022',
 'L14_V023',
 'L14_V024',
 'L14_V025',
 'L14_V026',
 'L14_V027',
 'L14_V028',
 'L15_V001',
 'L15_V002',
 'L15_V003',
 'L15_V004',
 'L15_V005',
 'L15_V006',
 'L15_V007',
 'L15_V008',
 'L15_V009',
 'L15_V010',
 'L15_V011',
 'L15_V012',
 'L15_V013',
 'L15_V014',
 'L15_V015',
 'L15_V016',
 'L15_V017',
 'L15_V018',
 'L15_V019',
 'L15_V020',

In [6]:
# be filtered at here
for batch in batch_videos_dict.keys():
    batch_videos_dict[batch] = [video for video in batch_videos_dict[batch] if video[:3] in filter_L]

In [7]:
batch_videos_dict

{'batch1': [],
 'batch2': ['L13_V001',
  'L13_V002',
  'L13_V003',
  'L13_V004',
  'L13_V005',
  'L13_V006',
  'L13_V007',
  'L13_V008',
  'L13_V009',
  'L13_V010',
  'L13_V011',
  'L13_V012',
  'L13_V013',
  'L13_V014',
  'L13_V015',
  'L13_V016',
  'L13_V017',
  'L13_V018',
  'L13_V019',
  'L13_V020',
  'L13_V021',
  'L13_V022',
  'L13_V023',
  'L13_V024',
  'L13_V025',
  'L13_V026',
  'L13_V027',
  'L13_V028',
  'L13_V029',
  'L13_V030']}

In [9]:
for batch in batch_videos_dict.keys():
    for video in batch_videos_dict[batch]:
        path_save = os.path.join(os.getcwd(), '..', '..', 'data', 
                            batch, 'clip-features-14', video + ".npy") 
        if os.path.isfile(path_save):
            continue

        path_to_keyframe = os.path.join(os.getcwd(), '..', '..', 'data', 
                            batch, 'keyframes', 'keyframes_' + video[:3], video) 
        print(path_to_keyframe)

        embeddings = []
        image_files = [os.path.join(path_to_keyframe, f) for f in os.listdir(path_to_keyframe) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        image_files.sort()

        for image_path in image_files:
            print(image_path)
            image = Image.open(image_path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                image_features = model.get_image_features(**inputs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            embeddings.append(image_features.cpu().numpy().flatten())
        embeddings_array = np.array(embeddings)
        path_save = os.path.join(os.getcwd(), '..', '..', 'data', 
                            batch, 'clip-features-14', video + ".npy") 
        np.save(path_save, embeddings_array)

/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/001.jpg
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/002.jpg
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/003.jpg
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/004.jpg
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/005.jpg
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/006.jpg
/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/src/clip-version/../../data/batch2/keyframes/keyframes_L13/L13_V001/007.jpg
/Users/V

KeyboardInterrupt: 